# Reproduction of First-principles analysis of cross-resonance gate operation

Malekakhlagh et al. PRA 102, 042605

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import pickle
from multiprocessing import Pool
from sympy import Add, I, Mul, Pow, Symbol, pi, simplify
from sympy.physics.quantum import Commutator, IdentityOperator, HermitianOperator, TensorProduct
from symqudit.two_transmon_hamiltonian import (TwoTransmonHamiltonian, sort_block_diagonal,
                                               to_dict, from_dict, dict_product)
from symqudit.schrieffer_wolff_expansion import SWExpansion, integrate_exps
from symqudit.common import ketbra, get_expr_at_order

In [ ]:
Id = IdentityOperator()

def expand_term(term):
    c_facts, ops = term.args_cnc()
    for iop, op in list(enumerate(ops)):
        if isinstance(op, Pow):
            for _ in range(op.args[1] - 1):
                ops.insert(iop, op.args[0])

    op_prod = dict_product(op_dicts[ops[0]], op_dicts[ops[1]])
    for op in ops[2:]:
        op_prod = dict_product(op_prod, op_dicts[op])

    overall = Mul(*c_facts)

    coeffs, ops = [], []
    for ket, bra_dict in op_prod.items():
        for bra, coeff in bra_dict.items():
            if ket[1] is None:
                coeffs.append(overall * coeff)
                ops.append(TensorProduct(ketbra(ket[0], bra[0]), Id))
            else:
                coeffs.append(overall * coeff)
                ops.append(TensorProduct(ketbra(ket[0], bra[0]), ketbra(ket[1], bra[1])))

    return coeffs, ops

In [ ]:
cutoff = 4
c_params = (100., 0.1)
t_params = (104., 0.1)

In [ ]:
tth = TwoTransmonHamiltonian((c_params[0], t_params[0]), (c_params[1], t_params[1]))

h_dirac = tth.h_dirac(cutoff=cutoff).expand()
h_dirac = h_dirac.subs({tth.drive_freq: tth.qt.symbolic_eigenvalue(1).doit(), tth.drive_phase: pi})
h_dirac = tth.subs_delta(h_dirac).expand()

subs = {}
for transmon in [tth.qc, tth.qt]:
    for level in range(cutoff):
        nu = transmon._transition_amp(level, level + 1)
        subs[nu] = Symbol(nu._latex(None), real=True)

h_dirac = h_dirac.subs(subs)

sw = SWExpansion()
int_scale = Symbol('lambda', real=True, nonnegative=True)
h_i = HermitianOperator('H_I')
expansion = sw.expand(int_scale * h_i, int_scale, 3)

In [ ]:
op_dicts = {h_i: to_dict(h_dirac)}
op_expr = {}
states_expr = {}
g = {}
gdot = {}
hieff = {}

def calc_heff_at_order(order):
    expr = get_expr_at_order(expansion, int_scale, order) + sw.gdot_n[order]
    expr = expr.expand(commutator=True, tensorproduct=True).doit().expand()
    op_expr[order] = expr

    if order == 1:
        # No commutator for Order λ -> just substitute H_I
        states_expr[order] = op_expr[order].subs({h_i: h_dirac})
    else:
        if isinstance(expr, Add):
            with Pool() as pool:
                term_lists = pool.map(expand_term, expr.args)
        else:
            term_lists = [expand_term(expr)]

        coeffs = sum((c for c, _ in term_lists), [])
        ops = sum((o for _, o in term_lists), [])

        with Pool() as pool:
            coeffs = pool.map(simplify, coeffs)

        states_expr[order] = Add(*[c * o for c, o in zip(coeffs, ops)])

    hieff[order], gdot[order] = sort_block_diagonal(states_expr[order])
    g[order] = integrate_exps(gdot[order]).expand()
    op_dicts.update({sw.g_n[order]: to_dict(g[order]), sw.gdot_n[order]: to_dict(gdot[order])})

In [ ]:
## Order λ
calc_heff_at_order(1)

# Special substitution
expansion = expansion.subs({sw.gdot_n[1]: h_i})

## Order λ^2
calc_heff_at_order(2)

# Special substitution
o2 = HermitianOperator('O_2')
expansion = expansion.subs({Commutator(sw.g_n[1], h_i): -I * 2 * o2})
op_dicts[o2] = to_dict(states_expr[2])

# Order λ^3
calc_heff_at_order(3)

with open('first_principle_qutrit.pkl', 'wb') as out:
    pickle.dump((hieff, g, gdot), out)